In [30]:
import numpy as np
import pandas as pd
#import time dependencies
from datetime import datetime
import time
from datetime import date, timedelta
import datetime
#import statistics dependencies
from scipy.stats import linregress
#import matplotlib dependencies
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.lines as mlines
# Import the API key.
from config import EIA_key
import eia
# Import the requests library.
import requests

#### Start and End Date for Matplotlib

In [31]:
start_date = "2018-01-01"
end_date =  datetime.datetime.now()
# end_date= datetime.datetime.now()

# API Call for UK/Turkish/Spanish/South Korean/Japanese/Chinese/Italian/French Gas Imports

In [32]:
# UK API
API_Call_US=f"http://api.eia.gov/series/?api_key={EIA_key}&series_id=NG.NW2_EPG0_SWO_R48_BCF.W"
Series_Name_US= "Weekly Lower 48 States Natural Gas Working Underground Storage, Weekly"
Series_ID_US= "NG.NW2_EPG0_SWO_R48_BCF.W"
# Turkish API
API_Call_East=f"http://api.eia.gov/series/?api_key={EIA_key}&series_id=NG.NW2_EPG0_SWO_R31_BCF.W"
Series_Name_East= "Weekly East Region Natural Gas Working Underground Storage, Weekly"
Series_ID_East= "NG.NW2_EPG0_SWO_R31_BCF.W"
#Spanish API
API_Call_MidWest=f"http://api.eia.gov/series/?api_key={EIA_key}&series_id=NG.NW2_EPG0_SWO_R32_BCF.W"
Series_Name_MidWest= "Weekly Midwest Region Natural Gas Working Underground Storage, Weekly"
Series_ID_MidWest= "NG.NW2_EPG0_SWO_R32_BCF.W"
#South Korean API
API_Call_Mountain=f"http://api.eia.gov/series/?api_key={EIA_key}&series_id=NG.NW2_EPG0_SWO_R34_BCF.W"
Series_Name_Mountain= "Weekly Mountain Region Natural Gas Working Underground Storage, Weekly"
Series_ID_Mountain= "NG.NW2_EPG0_SWO_R34_BCF.W"
#Japanese API
API_Call_Pacific=f"http://api.eia.gov/series/?api_key={EIA_key}&series_id=NG.NW2_EPG0_SWO_R35_BCF.W"
Series_Name_Pacific= "Weekly Pacific Region Natural Gas Working Underground Storage, Weekly"
Series_ID_Pacific= "NG.NW2_EPG0_SWO_R35_BCF.W"
#Chinese API
API_Call_SCentral=f"http://api.eia.gov/series/?api_key={EIA_key}&series_id=NG.NW2_EPG0_SWO_R33_BCF.W"
Series_Name_SCentral= "Weekly South Central Region Natural Gas Working Underground Storage, Weekly"
Series_ID_SCentral= "NG.NW2_EPG0_SWO_R33_BCF.W"

In [33]:
API_Calls=[API_Call_US, API_Call_East, API_Call_MidWest, API_Call_Mountain, API_Call_Pacific, API_Call_SCentral]
Series_Names=[Series_Name_US, Series_Name_East, Series_Name_MidWest, Series_Name_Mountain, Series_Name_Pacific, Series_Name_SCentral]
Series_IDs=[Series_ID_US, Series_ID_East, Series_ID_MidWest, Series_ID_Mountain, Series_ID_Pacific, Series_ID_SCentral]

list_headers=['Weekly Lower 48 States Natural Gas Working Underground Storage, Weekly (Billion Cubic Feet)',
             'Weekly East Region Natural Gas Working Underground Storage, Weekly (Billion Cubic Feet)',
             'Weekly Midwest Region Natural Gas Working Underground Storage, Weekly (Billion Cubic Feet)',
             'Weekly Mountain Region Natural Gas Working Underground Storage, Weekly (Billion Cubic Feet)',
             'Weekly Pacific Region Natural Gas Working Underground Storage, Weekly (Billion Cubic Feet)',
             'Weekly South Central  Region Natural Gas Working Underground Storage, Weekly (Billion Cubic Feet)']
numbers=(1,2,3,4,5,6)
names=('US_Storage','EastCoastStorage','MidWestStorage','MountainStorage','PacificStorage','SouthCentralStorage')

In [ ]:
d={}
imports=[] 
bigdf = pd.DataFrame()
US_Storage=pd.DataFrame()
EastCoastStorage=pd.DataFrame()
MidWestStorage=pd.DataFrame()
MountainStorage=pd.DataFrame()
PacificStorage=pd.DataFrame()
SouthCentralStorage=pd.DataFrame()

for i,j,k,m,n,name in zip(API_Calls, Series_Names, Series_IDs, list_headers, numbers, names):
    API_Call=i
    Series_Name= j
    Series_ID= k
    #install EIA package--- pip install EIA_python
    #pip install pyOpenSSL to get around SSL error
    #these are the API keys I set up
    api_key = EIA_key
    api = eia.API(api_key)
    #import data but still need to clean it
    series_search = api.data_by_series(series=Series_ID)
    df = pd.DataFrame(series_search)
    # define a series and pass through api and Series_ID
    def retrieve_time_series(api, Series_ID):
        """
        Return the time series dataframe, based on API and unique Series ID
        """
        #Retrieve Data By Series ID 
        series_search = api.data_by_series(series=Series_ID)
        ##Create a pandas dataframe from the retrieved time series
        global df
        df = pd.DataFrame(series_search)
        return df
    # define another function main to run the script through
    def main():
        """
        Run main script
        """
        #Create EIA API using your specific API key
        api_key = EIA_key
        api = eia.API(api_key)
    #     #Declare desired series ID
    #     Series_ID="PET.WGTSTUS1.W"
        df=retrieve_time_series(api, Series_ID)
        #Print the returned dataframe df
        print(df)
    req = requests.get(i, verify = False)
    #Clean the dataframe by resetting the index because the date is automatically set as the index
    df.reset_index(inplace=True)
    #{original header: new name given}
    df.rename(columns={m:name}, inplace=True)
    #{index:Date}
    df.rename(columns={'index':'Date'}, inplace=True)
    #create a dataframe [[date, name of each country]]
    d=df[['Date', name]]
    #convert date to datetime[64]
    d['Date']=pd.to_datetime(d['Date'], errors='coerce')
    bigdf['Date']=d['Date']
    bigdf[name] = d[name]
bigdf

# Chinese Imports of Natural Gas From the US

In [ ]:
# plot crude storage and gasoline storage on one graph with two axes
fig, ax1=plt.subplots(frameon=True, figsize=(10,5))
ax1.set_xlabel('Date')
ax1.set_ylabel('Imports of US Natural Gas (tcf)')
ax1.plot(bigdf['Date'], bigdf['Chinese_Gas_Imports'], color='red')
ax1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
_=plt.xticks(rotation=45)
ax1.tick_params(axis='y')
ax1.set_title("Chinese Imports of US Natural Gas")
ax1.grid()
red_line = mlines.Line2D([], [], color='red', label='Chinese Gas Imports')
plt.legend(handles=[red_line], loc='upper center')
plt.savefig("Imports_By_Country_Images/Chinese_Gas_Imports.png", bbox_inches='tight')

# Japanese Gas Imports of Natural Gas From the US

In [ ]:
# plot crude storage and gasoline storage on one graph with two axes
fig, ax1=plt.subplots(frameon=False, figsize=(10,5))
ax1.set_xlabel('Date')
ax1.set_ylabel('Imports of US Natural Gas (tcf)')
ax1.plot(bigdf['Date'], bigdf['Japanese_Gas_Imports'], color='blue')
ax1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
_=plt.xticks(rotation=45)
# ax1.tick_params(axis='y')
ax1.set_title("Japanese Imports of US Natural Gas")
ax1.grid()
red_line = mlines.Line2D([], [], color='blue', label='Japanese Gas Imports')
plt.legend(handles=[red_line], loc='upper center')
plt.savefig("Imports_By_Country_Images/Jap_Gas_Imports.png", bbox_inches='tight')

# Korean/Japanese/Chinese Natural Gas Imports From the US

In [ ]:
# plot crude storage and gasoline storage on one graph with two axes
fig, ax=plt.subplots(frameon=False, figsize=(10,5))
ax.set_xlabel('Date')
ax.set_ylabel('Imports of US Natural Gas (tcf)')
ax.plot(bigdf['Date'], bigdf['Korean_Gas_Imports'], color='green')
ax.plot(bigdf['Date'], bigdf['Japanese_Gas_Imports'], color='blue')
ax.plot(bigdf['Date'], bigdf['Chinese_Gas_Imports'], color='red')
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
_=plt.xticks(rotation=45)
# ax1.tick_params(axis='y')
ax.set_title("Imports of US Natural Gas")
ax.grid()

green_line = mlines.Line2D([], [], color='green', label='Korean Gas Imports')
blue_line = mlines.Line2D([], [], color='blue', label='Japanese Gas Imports')
red_line = mlines.Line2D([], [], color='red', label='Chinese Gas Imports')

plt.legend(handles=[green_line, blue_line, red_line], loc='upper left')
plt.savefig("Imports_By_Country_Images/Korean_Gas_Imports.png", bbox_inches='tight')